# L14a: Natural Language Models Before Transformers
In this lecture, we'll look at natural language models before the advent of transformers. We'll discuss the limitations of these models and how they paved the way for the development of transformer architectures.

The key concepts of this lecture include:
* Fill me in

The sources for this lecture were:
* [Vaswani, Ashish, Noam M. Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser and Illia Polosukhin. “Attention is All you Need.” Neural Information Processing Systems (2017).](https://arxiv.org/abs/1706.03762)
* [Ramsauer, H., Schafl, B., Lehner, J., Seidl, P., Widrich, M., Gruber, L., Holzleitner, M., Pavlovi'c, M., Sandve, G.K., Greiff, V., Kreil, D.P., Kopp, M., Klambauer, G., Brandstetter, J., & Hochreiter, S. (2020). Hopfield Networks is All You Need. ArXiv, abs/2008.02217.](https://arxiv.org/abs/2008.02217)
* [Phuong, M., & Hutter, M. (2022). Formal Algorithms for Transformers. ArXiv, abs/2207.09238.](https://arxiv.org/abs/2207.09238)

___